# LangGraph Agent Demo

---

## Setup

In [ ]:
# Import the irrigation agent
from irrigation_agent import (
    IrrigationAgent, 
    IrrigationDecision,
    MockDatabase,
    MockSensorNetwork
)
import json
import random

# Initialize agent with 3 retry attempts
agent = IrrigationAgent(max_sensor_retries=3)

print("✓ Agent initialized successfully")
print(f"  Max sensor retries: {agent.max_sensor_retries}")

---

## Test Case 1: Normal Operation (Success)

**Scenario:** Field #12 exists, sensor returns valid reading

**Expected:** Agent makes irrigation decision based on moisture level vs. crop requirements

In [ ]:
# Set random seed for reproducible results
random.seed(42)

# Make decision for Field #12 (Tomato crop)
result = agent.decide_json(field_id=12)

# Display result
print("\n" + "="*80)
print("RESULT - Normal Operation")
print("="*80)
print(json.dumps(result, indent=2))

# Verify success
assert result['decision'] in ['IRRIGATE', 'DO_NOT_IRRIGATE'], "Decision should be irrigation-related"
assert result['current_moisture'] is not None, "Moisture reading should be present"
assert len(result['errors']) == 0, "No errors expected in success case"

print("\n✓ TEST PASSED - Normal operation successful")

---

## Test Case 2: Field Not Found

**Scenario:** Field #999 does not exist in database

**Expected:** MAINTENANCE_REQUIRED with error message

In [ ]:
# Try to query non-existent field
result = agent.decide_json(field_id=999)

# Display result
print("\n" + "="*80)
print("RESULT - Field Not Found")
print("="*80)
print(json.dumps(result, indent=2))

# Verify failure handling
assert result['decision'] == 'MAINTENANCE_REQUIRED', "Should require maintenance"
assert len(result['errors']) > 0, "Error list should not be empty"
assert 'not found' in result['reason'].lower(), "Reason should mention field not found"

print("\n✓ TEST PASSED - Field not found handled correctly")

---

## Test Case 3: Sensor Timeout with Retry

**Scenario:** Sensor fails to respond (timeout), triggering retry mechanism

**Expected:** Agent retries up to 3 times, then escalates to MAINTENANCE_REQUIRED

In [ ]:
# Save original sensor method
original_sensor = MockSensorNetwork.get_soil_moisture

# Create mock that always times out
timeout_count = 0

def mock_timeout_sensor(field_id):
    global timeout_count
    timeout_count += 1
    print(f"[MOCK SENSOR] Timeout #{timeout_count}")
    return None  # Simulate timeout

# Replace sensor with mock
MockSensorNetwork.get_soil_moisture = staticmethod(mock_timeout_sensor)

try:
    # Reset counter
    timeout_count = 0
    
    # Attempt decision
    result = agent.decide_json(field_id=12)
    
    # Display result
    print("\n" + "="*80)
    print("RESULT - Sensor Timeout with Retry")
    print("="*80)
    print(json.dumps(result, indent=2))
    
    # Verify retry behavior
    assert result['decision'] == 'MAINTENANCE_REQUIRED', "Should escalate to maintenance"
    assert result['sensor_attempts'] >= 3, f"Should retry at least 3 times (got {result['sensor_attempts']})"
    assert timeout_count >= 3, f"Mock sensor should be called 3+ times (got {timeout_count})"
    
    print(f"\n✓ TEST PASSED - Retried {timeout_count} times before escalation")
    
finally:
    # Restore original sensor
    MockSensorNetwork.get_soil_moisture = original_sensor

---

## Test Case 4: Sensor Hardware Error

**Scenario:** Sensor returns impossible value (-50.0%) indicating hardware malfunction

**Expected:** Immediate MAINTENANCE_REQUIRED (no retry for hardware errors)

In [ ]:
# Save original sensor method
original_sensor = MockSensorNetwork.get_soil_moisture

# Create mock that returns impossible value
def mock_hardware_error_sensor(field_id):
    print(f"[MOCK SENSOR] Hardware error - returning -50.0%")
    return -50.0  # Impossible moisture value

# Replace sensor with mock
MockSensorNetwork.get_soil_moisture = staticmethod(mock_hardware_error_sensor)

try:
    # Attempt decision
    result = agent.decide_json(field_id=12)
    
    # Display result
    print("\n" + "="*80)
    print("RESULT - Hardware Error")
    print("="*80)
    print(json.dumps(result, indent=2))
    
    # Verify error handling
    assert result['decision'] == 'MAINTENANCE_REQUIRED', "Should require maintenance"
    assert result['sensor_attempts'] == 1, "Should NOT retry for hardware errors"
    assert any('hardware' in err.lower() or 'impossible' in err.lower() 
               for err in result['errors']), "Error should mention hardware issue"
    
    print("\n✓ TEST PASSED - Hardware error detected and handled immediately")
    
finally:
    # Restore original sensor
    MockSensorNetwork.get_soil_moisture = original_sensor

---

## Test Case 5: Multiple Fields Comparison

**Scenario:** Test all available fields to see varying decisions

**Expected:** Different decisions based on each field's crop requirements and moisture levels

In [ ]:
# Set seed for reproducibility
random.seed(123)

# Test all available fields
fields_to_test = [1, 2, 12, 15, 20]
results = []

print("\n" + "="*80)
print("TESTING MULTIPLE FIELDS")
print("="*80)

for field_id in fields_to_test:
    print(f"\n--- Field #{field_id} ---")
    result = agent.decide_json(field_id=field_id)
    results.append(result)

# Display summary table
print("\n" + "="*80)
print("SUMMARY TABLE")
print("="*80)
print(f"{'Field ID':<10} {'Crop':<15} {'Moisture':<12} {'Decision':<20}")
print("-"*80)

for r in results:
    field_info = MockDatabase.get_field_info(r['field_id'])
    crop = field_info.crop_type.value if field_info else 'N/A'
    moisture = f"{r['current_moisture']:.1f}%" if r['current_moisture'] else 'N/A'
    decision = r['decision']
    
    print(f"{r['field_id']:<10} {crop:<15} {moisture:<12} {decision:<20}")

print("\n✓ TEST PASSED - All fields processed successfully")

---

## Test Summary

All test cases demonstrate:

1. **Normal Operation** - Agent makes correct irrigation decisions
2. **Field Not Found** - Proper error handling for missing fields
3. **Sensor Timeout** - Retry mechanism works (3 attempts)
4. **Hardware Error** - Immediate escalation for impossible values
5. **Multiple Fields** - Handles diverse scenarios correctly

---

---